In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# read in datasets
full = pd.read_csv("sleep_quality_dataset.csv")
train_og = pd.read_csv("train.csv")
test_og = pd.read_csv("test.csv")

written answers are in PDF submission

## Problem 2

a) perform dataset preprocessing by removing columns that are non-numeric as well as the identifier column (ID). report which columns were removed.

In [3]:
# look at data to identify features to use
full.head()

,ID,Gender,Age,Occupation,Sleep_Duration,Sleep_Quality,Activity_Level,Stress_Level,BMI_Category,Blood_Pressure,Heart_Rate,Daily_Steps,Sleep_Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [4]:
# columns to remove
to_remove = ["ID", "Gender", "Occupation", "BMI_Category", "Blood_Pressure", "Sleep_Disorder"]

# remove from train and test
train = train_og.drop(columns=to_remove)
test = test_og.drop(columns=to_remove)

b) use an existing package (ex: sklearn) to train a multiple linear regression model on the training set using all the remaining features after part a. report the intercept and coefficients of the linear regression models and the following metrics on the training data: (1) MSE metric and (2) R^2 metric.

In [5]:
# set features
features = ["Age", "Sleep_Duration", "Activity_Level", "Stress_Level", "Heart_Rate", "Daily_Steps"]

x_train = train[features]
y_train = train["Sleep_Quality"]

# fit the model
skmodel = LinearRegression()
skmodel.fit(x_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [6]:
# intercept
print("intercept:", round(skmodel.intercept_, 3))

# coefficients
for feature, coefficient in zip(features, skmodel.coef_):
    print(f"{feature} coefficient: {round(coefficient, 3)}")

intercept: 4.614
Age coefficient: 0.014
Sleep_Duration coefficient: 0.664
Activity_Level coefficient: -0.001
Stress_Level coefficient: -0.322
Heart_Rate coefficient: -0.021
Daily_Steps coefficient: 0.0


In [7]:
# predictions
train_pred = skmodel.predict(x_train)

# mse
train_mse = mean_squared_error(y_train, train_pred)
print("train MSE:", round(train_mse, 3))

# r2
train_r2 = r2_score(y_train, train_pred)
print("train R^2:", round(train_r2, 3))

train MSE: 0.131
train R^2: 0.906


c) evaluate the model on the testing set. report the MSE and R^2 metrics on the testing set.

In [8]:
x_test = test[features]
y_test = test["Sleep_Quality"]

# predictions
test_pred = skmodel.predict(x_test)

# mse
test_mse = mean_squared_error(y_test, test_pred)
print("test MSE:", round(test_mse, 3))

# r2
test_r2 = r2_score(y_test, test_pred)
print("test R^2:", round(test_r2, 3))

test MSE: 0.129
test R^2: 0.913


d) now to improve model performance. choose two of the non-numeric columns you removed in part a and encode them numerically in a way that makes sense for analysis. explain your choice of features and encoding method. retrain your model including these additional features and report the new MSE and R^2 metrics on the training and testing sets.

In [9]:
# copy train and test datasets
train_enc = train_og.copy()
test_enc = test_og.copy()

# split blood pressure into systolic/diastolic
train_bp = train_enc["Blood_Pressure"].str.split("/", expand=True)
test_bp = test_enc["Blood_Pressure"].str.split("/", expand=True)

train_enc["Systolic_BP"] = pd.to_numeric(train_bp[0])
train_enc["Diastolic_BP"] = pd.to_numeric(train_bp[1])

test_enc["Systolic_BP"] = pd.to_numeric(test_bp[0])
test_enc["Diastolic_BP"] = pd.to_numeric(test_bp[1])

# encode gender (1 = male, 0 = female)
train_enc["Gender"] = (train_enc["Gender"].str.lower() == "male").astype(int)
test_enc["Gender"]  = (test_enc["Gender"].str.lower() == "male").astype(int)


In [10]:
# set features (adjusted)
features_d = ["Age", "Sleep_Duration", "Activity_Level", "Stress_Level", "Heart_Rate", "Daily_Steps",
            "Systolic_BP", "Diastolic_BP", "Gender"]

x_train_d = train_enc[features_d]
y_train_d = train_enc["Sleep_Quality"]

x_test_d = test_enc[features_d]
y_test_d = test_enc["Sleep_Quality"]

# fit the model
skmodel_d = LinearRegression()
skmodel_d.fit(x_train_d, y_train_d)

# predictions
train_pred_d = skmodel_d.predict(x_train_d)
test_pred_d = skmodel_d.predict(x_test_d)

# mse and r2
print("train MSE:", round(mean_squared_error(y_train_d, train_pred_d), 3))
print("train R^2:", round(r2_score(y_train_d, train_pred_d), 3))
print("test MSE:", round(mean_squared_error(y_test_d, test_pred_d), 3))
print("test R^2:", round(r2_score(y_test_d, test_pred_d), 3))


train MSE: 0.109
train R^2: 0.922
test MSE: 0.1
test R^2: 0.932


In [11]:
# also look at coefficients
for feature, coefficient in zip(features_d, skmodel_d.coef_):
    print(f"{feature} coefficient: {round(coefficient, 3)}")

Age coefficient: 0.032
Sleep_Duration coefficient: 0.467
Activity_Level coefficient: 0.003
Stress_Level coefficient: -0.385
Heart_Rate coefficient: -0.009
Daily_Steps coefficient: 0.0
Systolic_BP coefficient: 0.045
Diastolic_BP coefficient: -0.089
Gender coefficient: 0.143


e) interpret results (answer in PDF)

## Problem 3

a) implement the closed-form solution for multiple linear regression using matrix operations and train a model on the training set (using the dataset from problem 2b). write a function to predict the response on a new testing point. briefly explain your process.

In [12]:
# for closed-form, need to use numpy arrays
x_train = train[features].to_numpy(dtype=float)
y_train = train["Sleep_Quality"].to_numpy(dtype=float).reshape(-1, 1)
x_test = test[features].to_numpy(dtype=float)
y_test = test["Sleep_Quality"].to_numpy(dtype=float).reshape(-1, 1)

# add intercept column (of 1s)
xb = np.c_[np.ones((x_train.shape[0], 1)), x_train]

# compute theta
theta = np.linalg.pinv(xb.T @ xb) @ (xb.T @ y_train)

In [13]:
# function
def predict_closed_form(theta, x):
    '''
    theta: vector with intercept and feature coefficients
    x: vector of feature values (new testing point)
    returns the prediction
    '''
    x = np.asarray(x, dtype=float).reshape(1, -1)
    xb = np.c_[np.ones((x.shape[0], 1)), x]
    return (xb @ theta)[0, 0]

b) compare the models given by your implementation with those trained in problem 2. report the MSE and R^2 metrics for the models you implemented on both training and testing sets and compare the metrics to the ones from problem 2. discuss if the results of your implementation are similar to the previous results.

In [14]:
# predict on the training set
train_preds = []
for i in range(x_train.shape[0]):
    train_preds.append(predict_closed_form(theta, x_train[i]))
train_preds = np.array(train_preds)

# predict on the test set
test_preds = []
for i in range(x_test.shape[0]):
    test_preds.append(predict_closed_form(theta, x_test[i]))
test_preds = np.array(test_preds)

# mse and r2
train_mse = mean_squared_error(y_train.flatten(), train_preds)
train_r2 = r2_score(y_train.flatten(), train_preds)
test_se = mean_squared_error(y_test.flatten(), test_preds)
test_r2 = r2_score(y_test.flatten(), test_preds)

print("train MSE:", round(train_mse, 3))
print("train R^2:", round(train_r2, 3))
print("test MSE:", round(test_mse, 3))
print("test R^2:", round(test_r2, 3))

train MSE: 0.131
train R^2: 0.906
test MSE: 0.129
test R^2: 0.913


## Problem 4

a) consider feature X and response variable Y. implement a polynomial regression model that fits a polynomial of degree p to the sleep quality data using the least-square method. if p=2, the method will use two features (X and X^2), if p=3, the model will use 3 features (X, X^2, X^3), and so on for larger values of p. use your own implementation of the closed-form solution from problem 3 and adapt it to polynomial regression. briefly explain your design.

In [15]:
# degree (using p for now)
p = 3

# use single feature
x_train_1feature = train["Sleep_Duration"].to_numpy(dtype=float)
y_train = train["Sleep_Quality"].to_numpy(dtype=float).reshape(-1, 1)
x_test_1feature = test["Sleep_Duration"].to_numpy(dtype=float)
y_test = test["Sleep_Quality"].to_numpy(dtype=float).reshape(-1, 1)

# polynomial feature matrix
x_poly_train = np.column_stack([x_train_1feature**k for k in range(1, p+1)])

# add intercept column
xb_poly = np.c_[np.ones((x_poly_train.shape[0], 1)), x_poly_train]

# compute theta
theta_poly = np.linalg.pinv(xb_poly.T @ xb_poly) @ (xb_poly.T @ y_train)

In [16]:
# test prediction
x_new = 7.0 # sleep duration

# convert to polynomial feature vector
x_new_poly = [x_new**k for k in range(1, p+1)]

# use previous function
y_hat = predict_closed_form(theta_poly, x_new_poly)
y_hat

np.float64(7.291577972355299)

b) consider the house sleep quality prediction with feature X = sleep_duration. train a polynomial regression model for different values of p <= 5 using your implementation. include a table with the MSE and R^2 metrics on both the training and testing data for at least 3 different values of p. discuss your observations on how the MSE and R^2 metrics change with the degree of the polynomial.

In [17]:
# already have x_train_1feature, y_train, x_test_1feature, and y_test from before

# degrees to test
degrees = [2, 3, 4, 5]

# to store results
rows = []

# test with each p
for p in degrees:
    # polynomial feature matrices
    x_poly_train = np.column_stack([x_train_1feature**k for k in range(1, p+1)])
    x_poly_test = np.column_stack([x_test_1feature**k for k in range(1, p+1)])

    # add intercept
    xb_train = np.c_[np.ones((x_poly_train.shape[0], 1)), x_poly_train]
    xb_test = np.c_[np.ones((x_poly_test.shape[0], 1)), x_poly_test]

    # theta
    theta = np.linalg.pinv(xb_train.T @ xb_train) @ (xb_train.T @ y_train)

    # predictions
    yhat_train = (xb_train @ theta).flatten()
    yhat_test = (xb_test @ theta).flatten()

    # mse and r2
    train_mse = mean_squared_error(y_train.flatten(), yhat_train)
    train_r2 = r2_score(y_train.flatten(), yhat_train)
    test_mse = mean_squared_error(y_test.flatten(), yhat_test)
    test_r2 = r2_score(y_test.flatten(), yhat_test)

    rows.append([p, train_mse, train_r2, test_mse, test_r2])

results_df = pd.DataFrame(rows, columns=["p", "Train MSE", "Train R^2", "Test MSE", "Test R^2"])
results_df.round(3)

,p,Train MSE,Train R^2,Test MSE,Test R^2
0,2,0.300,0.786,0.351,0.763
1,3,0.292,0.791,0.331,0.776
2,4,0.291,0.792,0.329,0.777
3,5,0.290,0.793,0.328,0.778


c) what happens to the training and testing MSE as p increase, and why is this the case? (answer in PDF)

## Problem 5

a) write your own code for gradient descent for training multiple linear regression using the algorithm from class. first try without normalizing the columns, what happens? it is fine if you get an error in this step. next try normalizing each column by centering around the mean and dividing by the standard deviation. briefly explain your design.

In [18]:
# function for gradient descent
def gradient_descent(xb, y, alpha, num_iters):
    '''
    xb: matrix including intercept column of ones
    y: true target values
    alpha: learning rate
    num_iters: number of iterations
    returns theta
    '''
    n, d = xb.shape
    theta = np.zeros((d, 1))

    for i in range(num_iters):
        y_hat = xb @ theta
        grad = (2/n) * (xb.T @ (y_hat - y))
        theta = theta - alpha * grad

    return theta

In [19]:
# get x_train, y_train, x_test, and y_test again
x_train = train[features].to_numpy(dtype=float)
y_train = train["Sleep_Quality"].to_numpy(dtype=float).reshape(-1, 1)
x_test = test[features].to_numpy(dtype=float)
y_test = test["Sleep_Quality"].to_numpy(dtype=float).reshape(-1, 1)

# add intercept column
xb_train = np.c_[np.ones((x_train.shape[0], 1)), x_train]
xb_test = np.c_[np.ones((x_test.shape[0], 1)),  x_test]

In [20]:
# try without normalization
alpha = 0.1
num_iters = 100

theta_gd_no_norm = gradient_descent(xb_train, y_train, alpha, num_iters)
theta_gd_no_norm

/var/folders/25/n3_xcsh95_b9q35q4zd246_80000gn/T/ipykernel_41941/3650168883.py:14: RuntimeWarning: overflow encountered in matmul
  y_hat = xb @ theta
/var/folders/25/n3_xcsh95_b9q35q4zd246_80000gn/T/ipykernel_41941/3650168883.py:16: RuntimeWarning: invalid value encountered in subtract
  theta = theta - alpha * grad


array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]])

In [21]:
# now normalize and rerun

# compute mean and standard deviation
mean_x = x_train.mean(axis=0, keepdims=True)
stdev_x = x_train.std(axis=0, keepdims=True, ddof=0)

# normalize
x_train_norm = (x_train - mean_x) / stdev_x
x_test_norm = (x_test - mean_x) / stdev_x

# add intercept back
xb_train_norm = np.c_[np.ones((x_train_norm.shape[0], 1)), x_train_norm]
xb_test_norm = np.c_[np.ones((x_test_norm.shape[0], 1)),  x_test_norm]

theta_gd_no_norm = gradient_descent(xb_train_norm, y_train, alpha, num_iters)
theta_gd_no_norm

array([[ 7.37      ],
       [ 0.12127571],
       [ 0.52364508],
       [ 0.01926168],
       [-0.54057089],
       [-0.11007669],
       [ 0.11320534]])

b) vary the value of the learning rate (at least 3 different values including 0.01, 0.1, 0.5) and report the train MSE after different numbers of iterations (10, 50, 100). include in a table the MSE and R^2 metrics on the training and testing set after different numbers of iterations with the three choices of the learning rate.

In [22]:
# function to predict
def predict(xb, theta):
    return (xb @ theta).flatten()

In [23]:
# to test
alphas = [0.01, 0.1, 0.5]
iterations = [10, 50, 100]

# to store results
rows = []

# loop through alphas and iterations
for alpha in alphas:
    for num_iters in iterations:
        # run gradient descent
        theta = gradient_descent(xb_train_norm, y_train, alpha, num_iters)

        # get y predictions
        yhat_train = predict(xb_train_norm, theta)
        yhat_test = predict(xb_test_norm, theta)

        # calculate mse and r^2
        train_mse = mean_squared_error(y_train.flatten(), yhat_train)
        train_r2 = r2_score(y_train.flatten(), yhat_train)
        test_mse = mean_squared_error(y_test.flatten(), yhat_test)
        test_r2 = r2_score(y_test.flatten(), yhat_test)

        rows.append([alpha, num_iters, train_mse, train_r2, test_mse, test_r2])

gd_results = pd.DataFrame(
    rows,
    columns=["Alpha", "# Iterations", "Train MSE", "Train R^2", "Test MSE", "Test R^2"]
).sort_values(["Alpha", "# Iterations"])

# to display table results (USED GENAI FOR THIS AND ONLY THIS)
# makes it so only some results use scientific notation. 
# previously all were in scientific notation even if they didn't need to be
pd.options.display.float_format = (
    lambda x: f"{x:.3e}" if (abs(x) >= 1e4 or abs(x) < 1e-3) else f"{x:.3f}"
)

gd_results.round(3)

,Alpha,# Iterations,Train MSE,Train R^2,Test MSE,Test R^2
0,0.010,10,36.862,-25.335,34.777,-22.500
1,0.010,50,7.358,-4.257,7.202,-3.866
2,0.010,100,1.095,0.218,1.073,0.275
3,0.100,10,0.766,0.453,0.748,0.495
4,0.100,50,0.133,0.905,0.128,0.914
5,0.100,100,0.131,0.906,0.128,0.913
6,0.500,10,8683.347,-6202.425,9328.417,-6302.358
7,0.500,50,2.661e+19,-1.901e+19,2.858e+19,-1.931e+19
8,0.500,100,6.067e+38,-4.334e+38,6.517e+38,-4.403e+38


c) write some observations about the behavior of the algorithm (answer in PDF)